# Data Access Notebook

In this notebook, we’ll be walking through how to retrieve climate data from `climakitae` through a series of real-world analyses. These include visualizing time-series temperature data, retrieving warming-level data, and calculating 1-in-X metrics—all designed to help you get familiar with how `climakitae` works in practice.

**Intended Application:** As a user, I want to **<span style="color:red">understand how new core in `climakitae` works and how I can use it for my own climate analyses.</span>**


**Runtime**: ~20 min. to run through the whole notebook. Modifications to different queries may increase/decrease the overall runtime.

In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import climakitae as ck 

# High-level details 
The AE data catalog has many different types of data. Our helper library `climakitae` attempts to make accessing and retrieving this data intuitive, as well as simplify climate and statistical analysis with the data down the line, by performing some data transformations as the data is retrieved.<br><br> To retrieve the data, you'll need to make some selections as to your climate variable, data resolution, location settings, and many other options. There are also several high-level options you'll need to set when selecting your data, detailed below: 

### Data type: Gridded or Stations
**Gridded**: Gridded (i.e. raster) climate data at various spatial resolutions.<br><br>
**Stations**: Gridded (i.e. raster) climate data at unique grid cell(s) corresponding to the central coordinates of the selected weather station(s). 
- This data is bias-corrected (i.e localized) to the exact location of the weather station using the historical in-situ data from the weather station(s). 
- This data is currently only available for dynamically downscaled air temperature data. 

### Scientific approach: Time or Warming Level
**Time**: We can retrieve the data using a traditional time-based approach that allows you to select historical data, future projections, or both, along with a time-slice of interest. 
- **Historical Climate** includes data from 1980-2014 simulated from the same GCMs used to produce the Shared Socioeconomic Pathways (SSPs). It will be automatically appended to a SSP time series when both are selected. Because this historical data is obtained through simulations, it represents average weather during the historical period and is not meant to capture historical timeseries as they occurred.
- **Historical Reconstruction** provides a reference downscaled <a href="https://www.ecmwf.int/en/about/media-centre/focus/2020/fact-sheet-reanalysis" style="color: blue;" target="_blank">reanalysis</a> dataset based on atmospheric models fit to satellite and station observations, and as a result will reflect observed historical time-evolution of the weather.
- Future projections are available for <a href="https://climatescenarios.org/primer/socioeconomic-development" style="color: blue;" target="_blank">greenhouse gas emission scenario (Shared Socioeconomic Pathway, or SSP)</a> SSP 2-4.5, SSP 3-7.0, and SSP 5-8.5 through 2100.

**Warming Level**: Retrieve data based on future global warming levels. This method automatically gathers all available model data across the combined historical and future periods, then determines the time window during which each simulation reaches the selected warming level. 
- Because warming levels are defined by changes in global mean temperature, they allow for comparisons of potential outcomes across different scenarios and model simulations.
- Unlike a time-based approach, which limits analysis to simulations following a specific SSP trajectory, this method includes all simulations that reach the specified amount of warming, regardless of when that warming occurs.
<br>
### Downscaling method: Dynamical, Statistical, or both
**Dynamical**: <a href="https://dept.atmos.ucla.edu/alexhall/downscaling-cmip6" style="color: blue;" target="_blank">Dynamically-downscaled</a> WRF data, produced at hourly intervals. If you select `daily` or `monthly` for `Timescale`, you will receive an average of the hourly data. The spatial resolution options, on the other hand, are each the output of a different simulations, nesting to higher resolution over smaller areas.<br>

**Statistical**: <a href="https://loca.ucsd.edu" style="color: blue;" target="_blank">Hybrid-statistically downscaled</a> LOCA2-Hybrid data, available at daily and monthly timescales. Multiple LOCA2-Hybrid simulations are available (100+) at a fine spatial resolution of 3km.

# Data Selection Options

There are several methods to explore all the available data options in `climakitae`. You can get a comprehensive overview or explore step by step.

### Verbosity
You can also choose the level of output `climakitae` will provide for you. <br>
`-2` : errors only  
`-1` : warnings and errors  
`0`  : info, warnings, and errors (default)  
`1`  : debug, info, warnings, and errors (developers or debugging only, not recommended)

In [ ]:
# Initialize the interface
cd = ck.ClimateData(verbosity=-1) # only give error messages, quiet output

In [ ]:
# Get a comprehensive overview of all available options
cd.show_all_options()

# Or you can see specific categories by uncommenting any of the lines below:
# cd.show_catalog_options()
# cd.show_activity_id_options()
# cd.show_institution_id_options()
# cd.show_source_id_options()
# cd.show_experiment_id_options()
# cd.show_table_id_options()
# cd.show_grid_label_options()
# cd.show_variable_options()
# cd.show_installation_options()
# cd.show_processors()
# cd.show_boundary_options()
# cd.show_station_options()

### This will be explored in more detail in the next section

## Data Selection Options for a particular set of inputs
You can explore options step by step, building your query as you learn about available data.

In [ ]:
# Explore options step by step
print("=== Available Catalogs ===")
cd.reset()
cd.show_catalog_options()

print("\n=== Choose 'renewable energy generation' catalog and explore installations ===")
renewables_explorer = cd.catalog("renewable energy generation")
renewables_explorer.show_installation_options()

print("\n=== Choose 'pv_utility' installation and explore variables ===")
pv_explorer = renewables_explorer.installation("pv_utility")
pv_explorer.show_variable_options()

You can also explore the climate data catalog:

In [ ]:
print("=== Climate Data Catalog ===")
cd.reset()
data_explorer = cd.catalog("cadcat")

print("\n=== WRF (Dynamical Downscaling) Variables ===")
wrf_explorer = data_explorer.activity_id("WRF")
wrf_explorer.show_variable_options(show_n=15)

At any point in building your query, you can check what parameters you've set:

In [ ]:
# Build a partial query and check its state
partial_query = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .experiment_id("historical")
    .table_id("mon")
    .grid_label("d01")
)

# Check what we've built so far
partial_query.show_query()

# # See what variable options are still available
print("\nAvailable variables for this query:")
partial_query.show_variable_options(show_n=5)

You can reset the interface to start a new query at any time:

In [ ]:
cd.reset()
print("Interface reset - ready for new query")

# Retrieving data 
The `ClimateData` interface allows you to chain method calls to build readable queries, and then retrieve the data easily in your query. 
<br><br>
Required components of the query depend on the data catalog you're interested in. In general, the required components for all queries are: 
- catalog
- variable 

### Example 1: Future air temperature data 
You can retrieve data using a dictionary query, or by chaining operations to the `ClimateData` object. Either is valid and will result in the same output data, so just use whichever method is most intuitive to you. 

#### Method 1: Chained operations (recommended)

In [ ]:
cd.reset()
climate_data = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .experiment_id("ssp370")
    .institution_id("UCLA")
    .table_id("mon")
    .grid_label("d02")
    .variable("t2")
).get()
climate_data

#### Method 2: Dictionary query

In [ ]:
# Define your query 
climate_query_dict = {
    "catalog": "cadcat", # Catalog name 
    "activity_id": "WRF", # Downscaling method 
    "institution_id": "UCLA", # Institution name
    "experiment_id": "ssp370", # Simulation
    "table_id": "mon", # Temporal resolution 
    "grid_label": "d02", # Grid resolution
    "variable_id": "t2" # Variable name 
}

# Load the query 
climate_query = ck.ClimateData(verbosity=-2).load_query(climate_query_dict)

# Retrieve the data
climate_data = climate_query.get()
climate_data

### Example 2: Renewable energy model data 
Note that the renewables catalog has an additional query option: `installation`. This indicates the energy generation method, a parameter that is only applicable for this particular catalog. 

In [ ]:
# Define your query 
renewables_query_dict = {
    "catalog": "renewable energy generation", # Catalog name 
    "experiment_id": "historical", # Model name 
    "table_id": "day", # Temporal resolution 
    "grid_label": "d03", # Grid resolution
    "variable_id": "cf", # Variable name 
    "installation": "pv_utility", # Renewables catalog only! 
    # "source_id": "MPI-ESM1-2-HR" # Optional: pick a simulation within the model 
}

# Load the query 
renewables_query = ck.ClimateData(verbosity=-2).load_query(renewables_query_dict)

# Retrieve the data
renewables_data = renewables_query.get()
renewables_data

<br>

# Working with Processors
You can further customize your data retrieval using `processors`, which perform operations on the data before it is returned to you. The available processors are: 

- **`concat`** - Concatenate datasets along specified dimensions, default behavior is to concatenate on "time" by combining historical and SSP datasets.
- **`filter_unadjusted_models`** - Remove or include unadjusted models (default: "yes" to remove).
- **`update_attributes`** - Updates the attributes of your dataset based on the processors applied.
- `time_slice` - Applies a time slice to the requested dataset.
- `warming_level` - Applies a global warming level approach (as separate from the default time based approach). Please see our guidance on the use of global warming levels.
- `clip` - Applies a spatial clipping to the requested dataset. Many types of spatial clipping are supported including point based, bounding box, user provided shape files, and built-in boundaries including states, CA counties, CA watersheds, CA electric and utilities areas, CA demand forecast zones, CA electric balancing authority areas, and CA census tracts.
- `convert_units` - converts the units of your dataset.
- `metric_calc` - applies metric calculations to your dataset such as min, max, mean, median, percentiles, and 1-in-X calculations. 
- `bias_adjust_model_to_station` - For working with gridded data bias adjusted to historical HADISD weather station data.
- `export` - Exports your requested dataset to a range of file formats.

The first three processors (bolded) are run by default every time that you retrieve data. Examples of other available processors can be found in the `climakitae` library documentation, or in other example notebooks. <br><br>
It's important to note that processors are applied as a **dictionary**. This enables you to add more than one processor to your chain of operations. 

## Processor Examples

To bring cohension between the processor examples we'll be looking at below, we'll be building towards a complete, realistic analysis using WRF data. Any dataset works, but for demonstration purposes it is useful to see how the puzzle pieces fit together using a consistent dataset.

### Processor Example 1: Concatenation along a specified dimension
By default, when historical data is retrieved in the same operation as future data, the historical data will be appended to the future data, giving a single timeseries. However, you can change this default behavior by setting the query to concatenate along the simulation `sim` dimension instead. This will return the historical and future data as separate simulations. Concatenating by `time` or by `sim` have unique benefits, and you'll need to decide which method is most appropriate for your analyses. 

In the returned data from the code below, future time periods for the historical simulation will be infilled with `NaN`, because the model has no data for that time period by definition. The same logic applies to the future simulations: any time in the past will be infilled with `NaN`. 

In [ ]:
# Default behavior of concatenating by `time`
cd.reset()
concat_by_time = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .experiment_id(["historical", "ssp370"]) # Retrieve historical and future data 
    .table_id("mon")
    .grid_label("d01")
    .variable("prec") # Precipitation 
    .processes({
        "concat": "time"  # Concatenate along simulation dimension
    })
    .get()
)
concat_by_time

In [ ]:
# Example behavior of concatenating by `sim`
cd.reset()
concat_by_sim = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .experiment_id(["historical", "ssp370"]) # Retrieve historical and future data 
    .table_id("mon")
    .grid_label("d01")
    .variable("prec") # Precipitation 
    .processes({
        # Concatenate along simulation dimension. You'll notice that the `sim` dim
        # now has `historical` and `ssp370` simulations stacked ontop of each other
        "concat": "sim" 
    })
    .get()
)
concat_by_sim

### Processor Example 2: Filter Unadjusted Models (only for WRF data)

By default, we filter out unadjusted WRF models. If you're interested in learning more about why, you can check out the section in our guidance where we talk about that <a href="https://analytics.cal-adapt.org/guidance/using_in_decision_making/#the-analytics-engine-hosts-a-priori-bias-corrected-as-well-as-non-bias-corrected-wrf-data---which-one-should-a-user-choose?" style="color: blue;" target="_blank">here.</a> However, if you'd like to see all models, you can pass in `filter_unadjusted_models: False` into the processors. **However, we highly recommend ONLY using the adjusted WRF models for analysis, since the unadjusted models have biases that have not been corrected.**

In [ ]:
cd.reset()
all_models = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        'filter_unadjusted_models': 'no'
    })
    .get()
)

# Here, you'll see that there are 8 WRF models, versus the example above that concatenating by time, which only had 5 models.
all_models

### Processor Example 3: Time Slicing

Assuming you don't want to do an analysis on the entire timeseries of data available, we provide a TimeSlice processor to subset the climate of interest into a smaller temporal subset. Just this time, we'll time slice some `LOCA2` data to see how those parameters look like. Below is an example of how to use it:

In [ ]:
cd.reset()
time_slice = (2030, 2050)
time_slicing = (cd
    .catalog("cadcat")
    .activity_id("LOCA2")    # Activity ID is LOCA2 
    .institution_id("UCSD")  # Different institution for LOCA2
    .table_id("day")
    .grid_label("d03")
    .variable("tasmax") # Max daily temp at 2 meters
    .processes({
        "time_slice": time_slice
    })
    .get()
)
time_slicing

In [ ]:
time_slicing

### Processor Example 4: Global Warming Levels (GWL)

Another approach you can take for your analysis is to use Global Warming Levels instead of time, so that you can compare simulations based on when they reach a certain level of warming rather than what their warming is at a certain point in time. To learn more, you can check out our GWL methodology notebook <a href="https://github.com/cal-adapt/cae-notebooks/blob/main/analysis/warming_level_methods.ipynb" style="color: blue;" target="_blank">here</a>.

In [ ]:
wrf_wl_data = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0], # Warming levels in °C
            # "warming_level_window": 12, # Optional: specify the window size (in years) for the warming level calculation
        }
    })
    .get()
)
wrf_wl_data

### Processor Example 5: Clipping to Boundaries

Now, you are more likely to have a specific area of interest rather than conducting an analysis over the entire WRF/LOCA2 grid. Using `climakitae`, there are many options for clipping to boundaries. They include:
#### Clipping to a user specified shape file:
- `"clip": "<path to geopandas readable shape file>"`

#### Clipping to user specified lat/lons
- `"clip": (lat0, lon0)`
- `"clip": [(lat0, lon0), (lat1, lon1), ..., (latN, lonN)]`

#### Clipping to a HADISD station
Use `ClimateData().show_station_options()` to see a list of all accepted stations. Please note that this method DOES NOT bias adjust your data using historical station data, it simply pulls the data from the nearest grid cell in the data you're requesting.  
- `"clip": "KBFL"`
- `"clip": "Bakersfield Meadows Field (KBFL)"`
- `"clip": ["KBFL", "KBLH", "KBUR"]`

#### Clipping to `climakitae` supported boundaries
The supported boundary types can be seen with `ClimateData().show_boundary_options()`. To see a comprehensive list of the types you can use `ClimateData().show_boundary_options("<type>")`. Please be advised that some of these lists (like census tracts) are immense and may be listed by their numerical code.  

- `"clip": "Los Angeles County"`
- `"clip": ["Alameda County", "Los Angeles County"]`
- `"clip": {"boundaries": ["Alameda County", "Los Angeles County"], "separated" = True}`
- `"clip": ["Alameda County", "City and County of San Francisco - Hetch Hetchy Water and Power"]`

**Note: You may also clip to multiple boundaries at the same time.** In this case, the union is returned by default, enabling a clean and intuitive plotting experience (shown below). If you’d prefer to preserve location as a dimension for comparative analysis, you can use the approach demonstrated in the third bullet of *Clipping to `climakitae` support boundaries* above: pass a dictionary with a `boundaries` key containing your list of boundaries and set `separated`: True. This produces a new dimension named after the first boundary type in the list. For example, applying this approach to the scenario in bullet 4 would create a dimension called `county`, since a county is the first boundary specified.

In [ ]:
# Let's take a look at all HADISD station options
cd.show_station_options()

In [ ]:
# Now, let's look at other boundaries that `climakitae` supports
cd.show_boundary_options()
cd.show_boundary_options("ca_counties")
cd.show_boundary_options("ious_pous")

**Now, let's clip our WRF WL data to a specific point (Downtown Los Angeles).**

In [ ]:
wl_clipped_data = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0], # Warming levels in °C
        },
        "clip": (34.05, -118.25)  # Clip to specified coordinates
    })
    .get()
)

wl_clipped_data

**Now, let's clip to multiple points at the same time and take a look at the resulting data object.**

In [ ]:
lat_lons = [
    (34.05, -118.25),  # Los Angeles, CA
    (37.77, -122.42),  # San Francisco, CA
]
wl_couple_points = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0], # Warming levels in °C
        },
        "clip": {
            "points": lat_lons,  # Clip to specified coordinates
            # Whether or not you want your clipped points to be in a separate dimension called `points`,
            # or if you want gridded data returned with NaN everywhere except selected points.
            "separated": True
        }
    })
    .get()
)

wl_couple_points

**We can additionally clip different boundaries together too. Below, we'll clip Alameda and Los Angeles Counties into one xr.DataArray and visualize the result.**

In [ ]:
clip_counties = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0], # Warming levels in °C
        },
        "clip": ["Alameda County", "Los Angeles County"]
    })
    .get()
)

clip_counties.isel(sim=0, warming_level=0, time_delta=0).t2.plot(x='lon', y='lat')

**Now, we can separate the two clipped regions like we did before using the `separated: True` parameter to `clip`, and see that `county` now exists as a dimension on the resulting DataArray object.**

In [ ]:
clip_counties_separated = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0], # Warming levels in °C
        },
        "clip": {
            "boundaries": ["Alameda County", "Los Angeles County"],
            "separated": True
        }
    })
    .get()
)

clip_counties_separated

### Processor Example 6: Unit Conversion

We can also pass in a processor to convert the units of our dataset. It's as simple as specifying the conversion you'd like to apply. If it fails, you'll be informed in the logs about available units to convert to and no unit conversion will be applied.

In [ ]:
# Here, we'll take the temperature data that's natively in Kelvin to be returned in Fahrenheit instead
degF_data = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2") # Temperature at 2 meters
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0], # Warming levels in °C
        },
        "clip": "Los Angeles County",
        "convert_units": "degF" # This will convert from Kelvin to Fahrenheit
    })
    .get()
)
degF_data

In [ ]:
# Here we'll see that the values are within the reasonable ranges of we'd see for Fahrenheit for Los Angeles County
degF_data.mean(dim='time_delta').isel(sim=0, warming_level=0).t2.plot(x="lon", y="lat")

### Processor Example 7: Metric Calculation

Some basic metric calculations have also been built in as processors, so that you can save yourself a few steps of calculating common metrics on your resulting dataset.

The basic options include: `min`, `mean`, `median`, `max`, and `percentiles` and are demonstrated below. 

1-in-X calculations have also been included in as a processor as a more complex processor.

#### Simple Metric Comparison Across Warming Levels

Let's start with a simple metric comparison. We'll evaluate the min, mean, and max across three global warming levels, and then plot all those in a matrix. 

In [ ]:
metrics = ['min', 'mean', 'max']
data = []
for metric in metrics:
    data.append(
        (cd.catalog("cadcat")
        .activity_id("WRF")
        .institution_id("UCLA")
        .table_id("day")
        .grid_label("d03")
        .variable("t2")
        .processes({
            "warming_level": {
                "warming_levels": [1.5, 2.0, 3.0],
            },
            "clip": "Humboldt County",
            "convert_units": "degF",
            "metric_calc": {
                "metric": metric,
                # NOTE: We have a `time_delta` dimension here because that's the time variable that's returned from a warming
                # level approach of retrieving data. If this was a time-based approach, that dimension would be named `time`.
                "dim": ["time_delta", "sim"]
                # note: don't average over warming level
            }
        })
        .get())
    )

In [ ]:
# Taking a look at the 3 warming levels
fig, axes = plt.subplots(3, 3, figsize=(15, 12), sharex=True, sharey=True)

gwls = [1.5, 2.0, 3.0][::-1]  # Reverse for plotting
for i, metric in enumerate(metrics):
    for j, gwl in enumerate(gwls):
        ax = axes[j, i]
        data[i].isel(warming_level=len(gwls) - 1 - j).t2.plot.contourf(
            ax=ax,
            y="lat",
            x="lon",
            cbar_kwargs={
                'label': f'Temperature (°F)',
            }, 
            levels=100,
            cmap='plasma',
            vmin=0,
            vmax=110
        )
        ax.set_title(f'{metric.capitalize()} at {gwl}°C Warming Level')
        
plt.show()

#### Advanced Metric Comparison Across Warming Levels

Now, let's try a slightly more advanced analysis. Let's defined a base level of warming of 1.2 degC as our reference period, and then calculate the 90th, 95th, and 98th percentile temperatures within this reference period. 

Then, we'll get the temperature for three global warming levels (1.5, 2.0, 3.0) and count the average number of days per year above the 90th, 95th, and 98th percentile reference period temperature for each warming level for each individual GCM. Then we'll average the number of days across GCMs and plot the results in a matrix.

In [ ]:
# Let's first find the 90th, 95th, and 98th percentiles over Humboldt County for a 1.2 degC reference warming level period.
percentiles = [90, 95, 98]
location = "Humboldt County"
ref_percentiles = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2")
    .processes({
        "warming_level": {
            "warming_levels": [1.2],
        },
        "clip": location,
        "convert_units": "degF",
        "metric_calc": {
            "percentiles": percentiles,
            "dim": ["time_delta"]
        }
    })
    .get()
)

In [ ]:
# Now, we'll get all the raw data from the future WLs and count the number of days each simulation is above the different 1.2 degC WL percentiles.
wrf_wl_data = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2")
    .processes({
        "warming_level": {
            "warming_levels": [1.5, 2.0, 3.0],
        },
        "clip": location,
        "convert_units": "degF",
    })
    .get()
)

In [ ]:
display(ref_percentiles)
display(wrf_wl_data)

First, we'll drop `EC-Earth3-Veg` from `wrf_wl_data`, since `EC-Earth3-Veg` is an incomplete, and thus dropped, WL at 1.2 degC.

In [ ]:
# Dropping `EC-Earth3-Veg` by only keeping the other sims
only_valid_wrf = wrf_wl_data.sel(sim=[
    'WRF_UCLA_TaiESM1_ssp370_day_d03_r1i1p1f1',
    # 'WRF_UCLA_EC-Earth3-Veg_ssp370_day_d03_r1i1p1f1',
    'WRF_UCLA_MIROC6_ssp370_day_d03_r1i1p1f1',
    'WRF_UCLA_EC-Earth3_ssp370_day_d03_r1i1p1f1',
    'WRF_UCLA_MPI-ESM1-2-HR_ssp370_day_d03_r3i1p1f1'
])

# Creating a DataArray from the reference percentiles
ref_percentile_da = ref_percentiles[["t2_p90", "t2_p95", "t2_p98"]].isel(warming_level=0).to_array(dim="percentiles").assign_coords(percentiles=percentiles)

# Comparing the temperature values to the percentiles
compared_da = (only_valid_wrf > ref_percentile_da).t2

In [ ]:
### Now, let's plot a comparison of the results

fig, axes = plt.subplots(3, 3, figsize=(15, 12), sharex=True, sharey=True)

# mask for the spatial coordinates so that we can preserve them after counting
spatial_mask = ref_percentile_da.isel(percentiles=0, sim=0).notnull()

gwls = [1.5, 2.0, 3.0][::-1]  # Reverse for plotting
for i, metric in enumerate(percentiles):
    for j, gwl in enumerate(gwls):
        ax = axes[j, i]
        
        days_exceeding = (compared_da.isel(warming_level=len(gwls) - 1 - j).sel(percentiles=metric).mean(dim='sim')).sum(dim='time_delta', skipna=False) / 30

        # preserve the masked county
        days_exceeding = days_exceeding.where(spatial_mask)

        # contour plot
        days_exceeding.plot.contourf(
            ax=ax,
            y="lat",
            x="lon",
            cbar_kwargs={'label': f'Avg Days Per Year Above Reference'},
            levels=100,
            cmap='plasma',
            vmin=0, vmax=99
        )
        ax.set_title(f'Avg Days / Year > {metric}th pctl T2 at {gwl}°C GWL')

plt.suptitle("Average Number of Days Above Reference Percentile Temp for several Global Warming Levels")
plt.show()

#### 1-in-X Metric Calculations

Finally, the last metric calculation that we provide out of the box with the `ClimateData` object are 1-in-X calculations. Below demonstrates an example of how you could calculate the **Maximum 1-in-10 and 1-in-50 Daily Air Temperatures for WRF data over Alameda County for a 1.5 degC warming world.**

**Parameter options for 1-in-X calculations:**
- `return_periods`: A list of return periods you're interested in calculating for your dataset
- `distribution`: One of the following distribution types: `["gev", "gumbel", "weibull", "pearson3", "genpareto", "gamma"]` (see more on scipy.stats documentation <a href="https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions" style="color: blue;" target="_blank">here</a>).
- `extremes_type`: `min` or `max`
- `event_duration`: (# of time, scale of time), i.e. `(1, 'day')`
- `block_size`: Block size in number of years, default is 1
- `goodness_of_fit_test`:  # Whether or not you want to see the p-values from the fitted distributions used to calculate the resulting 1-in-X return values, default is True.

In [ ]:
return_periods = [10, 100]
one_in_x_data = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .variable("t2")
    .processes({
        "warming_level": {
            "warming_levels": [2.0]
        },
        "clip": "Alameda County", 
        "convert_units": "degF",
        "metric_calc": {
            "one_in_x": {
                "return_periods": return_periods, # Looking at 1-in-10 and 1-in-100
                "distribution": "gev", # Default is `gev`
                "extremes_type": "max",
                "event_duration": (1, 'day'),
                "block_size": 1, # Block size in years, default is 1
                "goodness_of_fit_test": True
            }
        }
    })
    .get())

# Let's plot what the 1-in-X return values look like over Alameda County
for x in return_periods:
    fig,ax = plt.subplots()
    one_in_x_data.mean(dim='sim').sel(one_in_x=x).return_values.plot(
        cmap='plasma', 
        x='lon', 
        y='lat',
        cbar_kwargs={'label': f'1-in-{x} year Daily Max Temperature (°F)'}
    )
    ax.set_title(f'1-in-{x} Year Daily Max Temp averaged \nover all WRF sims for Alameda County')
    plt.show()

### Processor Example 9: Bias Adjust Model to Station

In case you want to station bias correct gridded climate model data to historical observations from weather stations, you can pass in your desired weather station to the `bias_adjust_model_to_station` processor that we provide:

In [ ]:
# Let's manually bias adjust our climate data to KSAC (the SAC airport weather station).
manual_bias_adjustment = (cd
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("1hr")
    .grid_label("d02")
    .variable("t2")
    .processes({
        "bias_adjust_model_to_station": {
            "stations": ['KSAC'],
            "historical_slice": (1980, 2014), # You can specify what historical period to bias correct for.
        }
    })
    .get()
)

manual_bias_adjustment

### Processor Example 10: Export

Now that we’ve walked through the processors above, you may want to take the data you’ve been working with and move it elsewhere. The `export` processor makes this straightforward.

In the example below, we take a small subset of data, apply a few processors, and then export the result using the following parameters:

        filename (str, optional): Base output filename without extension.
            Default: "dataexport"
        file_format (str, optional): Output file format. Supported values:
            "NetCDF", "Zarr", "CSV". Case-insensitive. Default: "NetCDF"
        mode (str, optional): Storage location for Zarr files.
            "local" saves to local filesystem, "s3" saves to AWS S3.
            Default: "local"
        separated (bool, optional): When exporting a collection of point datasets,
            whether to create separate files for each point. If True, each dataset
            gets its own file with either lat/lon or index suffix. If False, all
            items are exported with the base filename (unique suffixes added if needed).
            Ignored for single gridded datasets. Default: False
        location_based_naming (bool, optional): When separated=True and
            exporting point-based data, include lat/lon coordinates in filenames
            (e.g., filename_34-0N_118-0W.nc). If False, uses index numbers
            instead (e.g., filename_0.nc). Silently ignored for gridded datasets.
            Default: False
        export_method (str, optional): Controls what data to export. Options:
            "data": Export all provided data (default)
            "raw": Export only raw/unprocessed data
            "calculate": Export only calculated/processed data
            "both": Export both raw and calculated data to separate files
            "skip_existing": Skip export if file already exists
            "none": Skip export entirely
            Default: "data"
        raw_filename (str, optional): Custom filename for raw data when using
            export_method="raw" or "both". If not provided, uses
            "{filename}_raw". Default: None
        calc_filename (str, optional): Custom filename for calculated data when
            using export_method="calculate" or "both". If not provided, uses
            "{filename}_calc".
            Default: None
        filename_template (str, optional): Custom template for generating filenames.
            Supports placeholders: {filename}, {lat}, {lon}, {name}.
            Lat/lon placeholders only populated for single-point data.
            Example: "{name}_data_{lat}N_{lon}W".
            Default: None
        fail_on_error (bool, optional): If True, raise exceptions on export
            errors. If False, log warnings and continue.
            Default: True

In [ ]:
# Let's export all of Humboldt County's data into one file, since the whole county can just be one file
location = "Humboldt County"
export_data = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("mon")
    .grid_label("d03")
    .variable("t2")
    .processes({
        "warming_level": {
            "warming_levels": [1.2],
        },
        "clip": location,
        "convert_units": "degF",
        "export": {
            "filename": "gridded_export_data", # Filename that you want the data exported to
        }
    })
    .get()
)

In [ ]:
lat_lons = [
    (34.05, -118.25),  # Los Angeles, CA
    (37.77, -122.42),  # San Francisco, CA
]

In [ ]:
# Now, let's export this list of `lat_lons` from before into separate files.
export_data = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("mon")
    .grid_label("d03")
    .variable("t2")
    .processes({
        "warming_level": {
            "warming_levels": [1.2],
        },
        "clip": {
            "points": lat_lons,
            "separated": True,
        },
        "convert_units": "degF",
        "export": {
            "separated": True, # (ONLY FOR POINT-BASED CLIPPING) Whether or not you want different points to be separated into different files
            "location_based_naming": True, # Whether or not you want the lat/lons of different points passed 
            "filename": "point_export_data", # Filename that you want the data exported to
        }
    })
    .get()
)

We hope this notebook and `climakitae` help empower you in your future climate data analyses!

**If you have any issues running this notebook, or are running into any issues with any of these processors, or have general feedback about the behavior of the `ClimateData` object or any of its processors, please write an Issue on our Github repo here: https://github.com/cal-adapt/climakitae/issues. Thank you so much!**